In [8]:
import os
from dotenv import load_dotenv
from PIL import Imag
import time
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import requests
import time
import psutil

In [38]:
load_dotenv()
character_dir = os.getenv('CHARACTER_DIR')  # 환경 변수

In [13]:
def crop_transparent(image_path, output_path):
    # 이미지를 불러옴
    image = Image.open(image_path)
    
    # 알파 채널을 추출
    alpha = image.split()[-1]
    
    # 알파 채널의 데이터를 가져와서 사용된 부분(투명하지 않은 부분)의 경계를 계산
    bbox = alpha.getbbox()
    
    # 경계를 기준으로 이미지를 자름
    cropped_image = image.crop(bbox)
    
    # 결과 이미지를 저장
    cropped_image.save(output_path, format='PNG')

In [18]:
def process_folders(base_directory):
    for folder_name in os.listdir(base_directory):
        if folder_name == '.ipynb_checkpoints' or folder_name.isdigit():
            continue
        folder_path = os.path.join(base_directory, folder_name)
        if os.path.isdir(folder_path):
            # 폴더명 + ' 잠금 화면.png' 파일 경로 생성
            input_png_path = os.path.join(folder_path, f'{folder_name} 잠금 화면.png')
            output_png_path = os.path.join(folder_path, f'{folder_name} 잠금 화면.png')
            
            if os.path.exists(input_png_path):
                crop_transparent(input_png_path, output_png_path)
                print(f'Processed: {folder_name}.png')
            else:
                print(f'File not found: {folder_name}.png')

In [19]:
process_folders(character_dir)

Processed: 고양이.png
Processed: 드래곤.png
Processed: 레비아탄.png
Processed: 무스.png
Processed: 벨루가.png
Processed: 설표.png
Processed: 시라소니.png
Processed: 아나콘다.png
Processed: 알파카.png
Processed: 염소.png
Processed: 올빼미.png
Processed: 와이번.png
Processed: 원숭이.png
Processed: 일각고래.png
Processed: 쿠거.png
Processed: 타조.png
Processed: 토끼.png
Processed: 퍼핀.png
Processed: 펭귄.png
Processed: 허스키.png


In [22]:
def install_driver():  # 드라이버 연결
    chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
    s = Service(f'./{chrome_ver}/chromedriver.exe')
    
    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--ignore-certificate-errors')  # SSL 인증서 오류 무시
    options.add_argument('--ignore-ssl-errors')
            
    try:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)  # 드라이버있으면 로드
    except:
        chromedriver_autoinstaller.install(True)  # 없으면 설치 후 로드
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
            
    return driver

In [35]:
def autocrop_gif(driver, file_path, output_dir, output_filename):
    driver.get("https://ezgif.com/crop")

    try:
        print(f'Processing: {output_filename}')
        
        # 파일 업로드
        upload_input = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "new-image")))
        upload_input.send_keys(file_path)
        upload_form = driver.find_element(By.ID, "upload-form")
        upload_form.submit()
        print('File uploaded, waiting for processing')
        time.sleep(2)

        # 업로드가 완료될 때까지 기다림 (업로드 완료 후 페이지 로드 대기)
        WebDriverWait(driver, 600).until(EC.presence_of_element_located((By.ID, "coords")))
        print('File processed, ready for autocrop')
        time.sleep(2)
        
        # 'trim transparent pixels around the image' 체크
        trim_checkbox = driver.find_element(By.NAME, "autocrop")
        trim_checkbox.click()
        print('Trim transparent pixels checkbox checked')
        time.sleep(2)

        # 'Crop image!' 버튼 클릭
        crop_button = driver.find_element(By.XPATH, "//input[@value='Crop image!']")
        crop_button.click()
        print('Crop image button clicked, waiting for processing')
        time.sleep(2)
        
        # 처리 완료 후 다운로드 링크가 나타날 때까지 대기
        download_a_tag = WebDriverWait(driver, 600).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#output > table > tbody > tr > td:nth-child(14) > a'))
        )
        download_url = download_a_tag.get_attribute("href")
        print('Processing done, download link available:', download_url)
        time.sleep(2)

        # 다운로드된 파일을 저장할 경로 설정
        output_file_path = os.path.join(output_dir, output_filename)
        
        # 다운로드 링크에서 파일을 직접 다운로드
        response = requests.get(download_url)
        with open(output_file_path, 'wb') as f:
            f.write(response.content)

        print(f'Successfully processed: {output_filename}')
    finally:
        driver.delete_all_cookies()
        time.sleep(2)

In [34]:
def process_folders_gif(base_directory):
    for folder_name in os.listdir(base_directory):
        if folder_name == '.ipynb_checkpoints' or folder_name.isdigit():
            continue
        
        driver = install_driver()
        folder_path = os.path.join(base_directory, folder_name)
        
        if os.path.isdir(folder_path):
            input_gif_path = os.path.join(folder_path, f'{folder_name} 달리는 모습.gif')
            if os.path.exists(input_gif_path):
                output_filename = f'{folder_name} 달리는 모습.gif'
                autocrop_gif(driver, input_gif_path, folder_path, output_filename)  # 실행
            else:
                print(f'File not found: {input_gif_path}')
    
    driver.quit()  # 크롤러 종료

In [36]:
process_folders_gif(character_dir)

Processing: 벨루가 달리는 모습.gif
File uploaded, waiting for processing
File processed, ready for autocrop
Trim transparent pixels checkbox checked
Crop image button clicked, waiting for processing
Processing done, download link available: https://ezgif.com/save/ezgif-5-d9eb1497a4.gif
Successfully processed: 벨루가 달리는 모습.gif
Processing: 허스키 달리는 모습.gif
File uploaded, waiting for processing
File processed, ready for autocrop
Trim transparent pixels checkbox checked
Crop image button clicked, waiting for processing
Processing done, download link available: https://ezgif.com/save/ezgif-4-11b8ad80e2.gif
Successfully processed: 허스키 달리는 모습.gif
